<a href="https://colab.research.google.com/github/Nikhileswar-Komati/100D_ML/blob/master/BIGMART_SALES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
files.upload()

In [3]:
train_df = pd.read_csv('/content/BigMart_Train.csv')
test_df = pd.read_csv('/content/BigMart_Test.csv')
train_df['Source'] = 'Train'
test_df['Source'] = 'Test'
data = pd.concat([train_df, test_df])
print(train_df.shape, test_df.shape, data.shape)
data.sample(5)

(8523, 13) (5681, 12) (14204, 13)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Source
4683,FDL58,5.78,Regular,0.074451,Snack Foods,264.5568,OUT018,2009,Medium,Tier 3,Supermarket Type2,6327.7632,Train
7499,FDE56,NaN,Regular,0.278731,Fruits and Vegetables,63.2194,OUT019,1985,Small,Tier 1,Grocery Store,247.6776,Train
3821,FDM12,16.70,Regular,0.070026,Baking Goods,187.8214,OUT049,1999,Medium,Tier 1,Supermarket Type1,NaN,Test
1231,FDQ34,10.85,Low Fat,0.162495,Snack Foods,104.5622,OUT049,1999,Medium,Tier 1,Supermarket Type1,NaN,Test
5715,FDC22,6.89,Regular,0.136315,Snack Foods,195.0820,OUT013,1987,High,Tier 3,Supermarket Type1,2703.1480,Train


###MISSING-VALUES

In [4]:
impute_weights = data.pivot_table(values = 'Item_Weight', index = 'Item_Identifier')
count = 0
for i, row in data.loc[data['Item_Weight'].isnull(), :].iterrows():
  x = row['Item_Identifier']
  if x in impute_weights.index:
    count += 1
    data.loc[i,'Item_Weight'] = impute_weights.loc[x].values[0]
print("IMPUTED VALUES", count)



IMPUTED VALUES 2439


In [5]:
from scipy.stats import mode
impute_sizes = data.pivot_table(values = 'Outlet_Size', index = ['Outlet_Type', 'Outlet_Location_Type'], aggfunc = (lambda x: mode(x).mode[0]))
impute_sizes.loc[('Grocery Store', 'Tier 3')] = 'Small2'
count = 0
for i, row in data.loc[data['Outlet_Size'].isnull(), :].iterrows():
  x = (row['Outlet_Type'], row['Outlet_Location_Type'])
  if x in impute_sizes.index:
    count += 1
    data.loc[i,'Outlet_Size'] = impute_sizes.loc[x].values[0]
print("IMPUTED VALUES", count)

IMPUTED VALUES 4016
IMPUTED VALUES 0


In [7]:
impute_visibility = data.pivot_table(values = 'Item_Visibility', index = ['Item_Identifier'])
count = 0
for i, row in data.loc[data['Item_Visibility'] == 0, :].iterrows():
  x = row['Item_Identifier']
  if x in impute_visibility.index:
    count += 1
    data.loc[i,'Item_Visibility'] = impute_visibility.loc[x].values[0]
print("IMPUTED VALUES", count)

IMPUTED VALUES 879


In [8]:
data.loc[data['Item_Visibility'] == 0].shape

(0, 13)

In [9]:
data['Item_Visibility_MeanRatio'] = data.apply(lambda x: x['Item_Visibility'] / impute_visibility.loc[x['Item_Identifier']].values[0], axis=1)
data['Item_Visibility_MeanRatio'].describe()

count    14204.000000
mean         1.088248
std          0.503002
min          0.041552
25%          0.924160
50%          0.999070
75%          1.055881
max         14.575492
Name: Item_Visibility_MeanRatio, dtype: float64

In [12]:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [13]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [14]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})


In [15]:
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
data['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

In [17]:
#One Hot Coding:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet_Identifier'])

In [18]:
data.dtypes

Item_Identifier                       object
Item_Weight                          float64
Item_Visibility                      float64
Item_Type                             object
Item_MRP                             float64
Outlet_Establishment_Year              int64
Item_Outlet_Sales                    float64
Source                                object
Item_Visibility_MeanRatio            float64
Outlet_Years                           int64
Item_Fat_Content_Low Fat               uint8
Item_Fat_Content_Non-Edible            uint8
Item_Fat_Content_Regular               uint8
Outlet_Location_Type_Tier 1            uint8
Outlet_Location_Type_Tier 2            uint8
Outlet_Location_Type_Tier 3            uint8
Outlet_Size_High                       uint8
Outlet_Size_Medium                     uint8
Outlet_Size_Small                      uint8
Outlet_Size_Small2                     uint8
Outlet_Type_Grocery Store              uint8
Outlet_Type_Supermarket Type1          uint8
Outlet_Typ

In [19]:
data.head(3)

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Source,Item_Visibility_MeanRatio,Outlet_Years,Item_Fat_Content_Low Fat,Item_Fat_Content_Non-Edible,Item_Fat_Content_Regular,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Size_Small2,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Combined_Drinks,Item_Type_Combined_Food,Item_Type_Combined_Non-Consumable,Outlet_Identifier_OUT010,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049
0,FDA15,9.30,0.016047,Dairy,249.8092,1999,3735.1380,Train,0.931078,14,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,2009,443.4228,Train,0.933420,4,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,1999,2097.2700,Train,0.960069,14,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [20]:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

#Divide into test and train:
train = data.loc[data['Source']=="Train"]
test = data.loc[data['Source']=="Test"]

#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','Source'],axis=1,inplace=True)
train.drop(['Source'],axis=1,inplace=True)

#Export files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
